In [1]:
%cd ../

/home/dmoreno/pipeline_v4_final/pipeline/training/stamp_classifier/models/experimentation/stamp_full


In [2]:
import pandas as pd

In [3]:
normalized_ndarrays = pd.read_pickle('data/normalized_ndarrays_hasavro.pkl')
normalized_ndarrays.keys()

dict_keys(['x_train', 'pos_train', 'y_train', 'oid_train', 'x_val', 'pos_val', 'y_val', 'oid_val', 'x_test', 'pos_test', 'y_test', 'oid_test', 'label_encoder'])

In [5]:
print(len(normalized_ndarrays['oid_train']))
print(len(normalized_ndarrays['oid_test']))
print(len(normalized_ndarrays['oid_val']))

29109
9704
9703


In [6]:
len(normalized_ndarrays['oid_train']) + len(normalized_ndarrays['oid_test']) + len(normalized_ndarrays['oid_val'])

48516

In [4]:
metadata_hasavro = pd.read_parquet('data/full_stamp_classifier_metadata_hasavro.parquet')
metadata_hasavro

,oid,candid,ra,dec,gal_l,gal_b,ecl_long,ecl_lat,magpsf,sigmapsf,...,sgscore1,sgscore2,sgscore3,distpsnr1,distpsnr2,distpsnr3,ndethist,ncovhist,chinr,sharpnr
0,ZTF21aaebkmj,1501128131215015004,65.163495,-26.636435,224.888336,-43.699489,56.438373,-47.223415,19.800232,0.179855,...,0.500000,0.001250,0.500000,5.558571,6.079560,7.264775,12,232,3.333,0.240
1,ZTF18abtnlik,643114394415015053,279.321654,49.148389,78.016574,22.385199,289.985625,71.941418,20.333069,0.180506,...,0.001250,0.994167,0.997250,5.262714,6.867275,9.061376,150,960,2.963,0.086
2,ZTF19acjurot,1058126442015015014,320.853375,-17.720127,32.664194,-41.349474,317.674423,-2.295138,18.519318,0.107349,...,0.009167,0.500000,0.000000,0.008522,0.558093,7.119209,10,133,1.575,0.042
3,ZTF20aanvpzo,1200441200215015003,224.375994,8.389928,6.711000,54.797328,219.201834,24.147229,19.587193,0.166170,...,0.763083,0.500000,0.107167,8.462006,9.868366,10.590695,13,345,1.611,0.367
4,ZTF20aawxpru,1306411870015015009,322.840280,9.429956,62.842008,-29.301685,328.512798,22.789567,20.299837,0.208037,...,0.500000,0.254470,0.140304,5.226012,5.712976,8.442719,43,422,7.562,0.575
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49125,ZTF19acrcngf,1043073070615015017,287.896865,-13.058972,23.448670,-10.320624,287.658060,9.289962,17.936604,0.088779,...,0.646083,0.938863,0.694062,2.037887,2.453805,3.977600,1,599,0.484,-0.019
49126,ZTF19acrdiud,1043169350415015017,359.363247,2.152355,96.853290,-57.959728,0.272379,2.227941,18.803501,0.181385,...,0.001875,0.500000,0.033952,2.019713,4.380590,14.092403,1,334,3.329,0.248
49127,ZTF19acrdyfr,1043265262115015067,72.337484,54.630553,153.044004,6.326199,78.058172,31.921084,19.165157,0.193813,...,0.964786,0.995417,0.997417,3.916241,10.256860,12.908553,1,456,0.633,-0.025
49128,ZTF19acrqqvl,1044312843715015025,110.675771,48.877886,168.911325,25.030347,105.040261,26.512481,16.571047,0.050797,...,0.145137,0.229018,0.988542,2.520166,8.693634,14.109303,1,371,7.370,0.283


In [7]:
metadata_hasavro['non_detections'] = metadata_hasavro['ncovhist'] - metadata_hasavro['ndethist']
metadata_hasavro = metadata_hasavro.fillna(-999)
clipping_rules = {
    'sgscore1': (-1, None),
    'distpsnr1': (-1, None),
    'sgscore2': (-1, None),
    'distpsnr2': (-1, None),
    'sgscore3': (-1, None),
    'distpsnr3': (-1, None),
    'fwhm': (None, 10),
    'ndethist': (None, 20),
    'ncovhist': (None, 3000),
    'chinr': (-1, 15),
    'sharpnr': (-1, 1.5),
    'non_detections': (None, 2000)
}
for feature, (min_val, max_val) in clipping_rules.items():
    if feature in metadata_hasavro.columns:
        if min_val is not None:
            metadata_hasavro[feature] = metadata_hasavro[feature].clip(lower=min_val)
        if max_val is not None:
            metadata_hasavro[feature] = metadata_hasavro[feature].clip(upper=max_val)

oids_train = normalized_ndarrays['oid_train']
oids_val = normalized_ndarrays['oid_val']
oids_test = normalized_ndarrays['oid_test']

print(oids_train.shape)
print(oids_val.shape)
print(oids_test.shape)

train_metadata_hasavro = metadata_hasavro.drop_duplicates(subset='oid', keep='first').set_index('oid').loc[oids_train]
val_metadata_hasavro = metadata_hasavro.drop_duplicates(subset='oid', keep='first').set_index('oid').loc[oids_val]
test_metadata_hasavro = metadata_hasavro.drop_duplicates(subset='oid', keep='first').set_index('oid').loc[oids_test]

print(train_metadata_hasavro.shape)
print(val_metadata_hasavro.shape)
print(test_metadata_hasavro.shape)

features = metadata_hasavro.columns.difference(['oid', 'candid'])

# 1. Calcular stats del set de entrenamiento
means = train_metadata_hasavro[features].mean()
stds = train_metadata_hasavro[features].std().replace(0, 1)

# 2. Aplicar normalización a todos los conjuntos
train_metadata_hasavro[features] = (train_metadata_hasavro[features] - means) / stds
val_metadata_hasavro[features]   = (val_metadata_hasavro[features]   - means) / stds
test_metadata_hasavro[features]  = (test_metadata_hasavro[features]  - means) / stds

final_metadata_hasavro = pd.concat([train_metadata_hasavro, val_metadata_hasavro, test_metadata_hasavro])
final_metadata_hasavro.to_parquet('data/clipped_normalized_metadata_hasavro.parquet')

(29109,)
(9703,)
(9704,)
(29109, 24)
(9703, 24)
(9704, 24)
